# 학습 모델 저장하고 재사용하기

In [56]:
# 데이터를 읽어 들이고 변환
import tensorflow as tf
import numpy as np

data = np.loadtxt('../Neural-Network-Implementation/data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [57]:
# 신경망 모델 구성
# global_step : 학습 횟수를 카운트하는 변수
global_step = tf.Variable(0, trainable=False, name='global_step')

In [58]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

# 신경망 계층 늘리기
# [10, 20] : 앞단 계층의 출력 크기 10, 뒷단 계층의 입력 크기 20
W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

# 최적화할 때마다 global_step 변수 값을 1씩 증가시킨다.
train_op = optimizer.minimize(cost, global_step=global_step)

In [59]:
# 모델 불러오기
sess = tf.Session()
# tf.global_variables() : 앞서 정의한 변수들을 가져오는 함수
saver = tf.train.Saver(tf.global_variables())

In [60]:
# ./model 디렉토리에 기존에 학습해둔 모델이 있는지 확인
ckpt = tf.train.get_checkpoint_state('./model')
# 체크포인트 파일에 학습한 모델이 저장되어있는지 확인
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    # 학습된 값들을 불러온다.
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    # 존재하지 않으면 변수를 새로 초기화한다.
    sess.run(tf.global_variables_initializer())

In [61]:
# 최적화를 수행한다.
for step in range(4):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' %sess.run(global_step),
         'Cost: %.3f' %sess.run(cost, feed_dict={X: x_data, Y:y_data}))

Step: 1,  Cost: 1.091
Step: 2,  Cost: 1.049
Step: 3,  Cost: 1.014
Step: 4,  Cost: 0.980


In [62]:
# 학습된 변수들을 지정한 체크포인트 파일에 저장한다.
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-4'

In [63]:
# 예측 결과와 정확도 확인
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값: ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값: ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값:  [0 1 2 0 0 0]
실제값:  [0 1 2 0 0 2]
정확도: 83.333328


# 텐서보드 사용하기

In [64]:
# with tf.name_scope 로 묶은 블록은 텐서보드에서 한 계층 내부를 표현해준다.
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
with tf.name_scope('layer2'):
    W1 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

In [65]:
# 손실값을 추적하기 위해 수집할 값을 지정하는 코드를 작성한다.
# tf.summary.scalar : 값이 하나인 텐서를 수집할 때 사용한다.
tf.summary.scalar('cost', cost)

<tf.Tensor 'cost_2:0' shape=() dtype=string>

In [67]:
# 모델 불러오기
sess = tf.Session()
# tf.global_variables() : 앞서 정의한 변수들을 가져오는 함수
saver = tf.train.Saver(tf.global_variables())

# ./model 디렉토리에 기존에 학습해둔 모델이 있는지 확인
ckpt = tf.train.get_checkpoint_state('./model2')
# 체크포인트 파일에 학습한 모델이 저장되어있는지 확인
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    # 학습된 값들을 불러온다.
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    # 존재하지 않으면 변수를 새로 초기화한다.
    sess.run(tf.global_variables_initializer())

In [68]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [69]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d' %sess.run(global_step),
         'Cost: %.3f' %sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 1 Cost: 1.013
Step: 2 Cost: 0.954
Step: 3 Cost: 0.906
Step: 4 Cost: 0.866
Step: 5 Cost: 0.830
Step: 6 Cost: 0.795
Step: 7 Cost: 0.766
Step: 8 Cost: 0.740
Step: 9 Cost: 0.718
Step: 10 Cost: 0.698
Step: 11 Cost: 0.681
Step: 12 Cost: 0.665
Step: 13 Cost: 0.650
Step: 14 Cost: 0.637
Step: 15 Cost: 0.625
Step: 16 Cost: 0.614
Step: 17 Cost: 0.605
Step: 18 Cost: 0.597
Step: 19 Cost: 0.590
Step: 20 Cost: 0.584
Step: 21 Cost: 0.579
Step: 22 Cost: 0.575
Step: 23 Cost: 0.571
Step: 24 Cost: 0.568
Step: 25 Cost: 0.565
Step: 26 Cost: 0.563
Step: 27 Cost: 0.561
Step: 28 Cost: 0.560
Step: 29 Cost: 0.558
Step: 30 Cost: 0.557
Step: 31 Cost: 0.556
Step: 32 Cost: 0.556
Step: 33 Cost: 0.555
Step: 34 Cost: 0.554
Step: 35 Cost: 0.554
Step: 36 Cost: 0.553
Step: 37 Cost: 0.553
Step: 38 Cost: 0.553
Step: 39 Cost: 0.552
Step: 40 Cost: 0.552
Step: 41 Cost: 0.552
Step: 42 Cost: 0.552
Step: 43 Cost: 0.551
Step: 44 Cost: 0.551
Step: 45 Cost: 0.551
Step: 46 Cost: 0.551
Step: 47 Cost: 0.551
Step: 48 Cost: 0.551
S

In [70]:
# tf.summary.merge_all : 앞서 지정한 텐서들을 수집한 다음
# tf.summary.FileWriter 함수를 이용해 그래프와 텐서들의 값을 저장할 디렉토리 설정
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs2', sess.graph)

In [71]:
# 최적화 실행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' %sess.run(global_step),
         'Cost: %.3f ' %sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    # sess.run 을 통해 merged로 모아둔 텐서의 값들을 계산하여 수집
    # writer.add_summary 함수를 통해 해당 값들을 앞서 지정한 디렉터리에 저장
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step: 101,  Cost: 0.550 
Step: 102,  Cost: 0.550 
Step: 103,  Cost: 0.550 
Step: 104,  Cost: 0.550 
Step: 105,  Cost: 0.550 
Step: 106,  Cost: 0.550 
Step: 107,  Cost: 0.550 
Step: 108,  Cost: 0.550 
Step: 109,  Cost: 0.550 
Step: 110,  Cost: 0.550 
Step: 111,  Cost: 0.550 
Step: 112,  Cost: 0.550 
Step: 113,  Cost: 0.550 
Step: 114,  Cost: 0.550 
Step: 115,  Cost: 0.550 
Step: 116,  Cost: 0.550 
Step: 117,  Cost: 0.550 
Step: 118,  Cost: 0.550 
Step: 119,  Cost: 0.550 
Step: 120,  Cost: 0.550 
Step: 121,  Cost: 0.550 
Step: 122,  Cost: 0.550 
Step: 123,  Cost: 0.550 
Step: 124,  Cost: 0.550 
Step: 125,  Cost: 0.550 
Step: 126,  Cost: 0.550 
Step: 127,  Cost: 0.550 
Step: 128,  Cost: 0.550 
Step: 129,  Cost: 0.550 
Step: 130,  Cost: 0.550 
Step: 131,  Cost: 0.550 
Step: 132,  Cost: 0.550 
Step: 133,  Cost: 0.549 
Step: 134,  Cost: 0.549 
Step: 135,  Cost: 0.549 
Step: 136,  Cost: 0.549 
Step: 137,  Cost: 0.549 
Step: 138,  Cost: 0.549 
Step: 139,  Cost: 0.549 
Step: 140,  Cost: 0.549 


In [73]:
saver.save(sess, './model2/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값: ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값: ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도: 100.00


In [74]:
tf.summary.histogram('Weights', W1)

<tf.Tensor 'Weights:0' shape=() dtype=string>